In [16]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [24]:
counties = "https://www.zillow.com/browse/homes/"
state = 'in'


request_headers = {
    'accept':
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip,deflate,br',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as session:
    url = counties + state + '/'
    response = session.get(url, headers = request_headers)


soup = BeautifulSoup(response.content, 'html.parser')

def newest(href):
    return href and re.compile('/newest-homes').search(href)

def other(href):
    return href and not re.compile('/newest-homes').search(href) and re.compile('-county/').search(href)

x = []
for a in soup.find_all(href = other):
    x.append(a.get('href'))

county_links = ["https://www.zillow.com/browse/homes/" + state + '/' + end for end in x]

In [34]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

def zip(href):
    return href and re.compile('([0-9]{5})').search(href)

for county in county_links:
    with requests.Session() as session:
        response = session.get(county, headers = req_headers)
    soup = BeautifulSoup(response.content,'html.parser')


    zips = []
    for a in soup.find_all(href = zip):
        zips.append(a.get('href'))
    


In [ ]:
zip_links = [county + z for county in county_links for z in zips]
print(zip_links)

In [83]:
def adds(href):
    return href and re.compile('([0-9])/').search(href) and not re.compile('[a-zA-Z]').search(href)

for zl in zip_links:
    with requests.Session() as session:
        response = session.get(zl, headers = request_headers)
    soup = BeautifulSoup(response.content,'html.parser')
    sections = []
    print(soup.find_all(href = adds))
    for a in soup.find_all(href = adds):
        print(a.get('href'))
        sections.append(a.get('href'))


[]
[]
[]
[]


KeyboardInterrupt: 

In [73]:
def homes(href):
    return href and re.compile('/home-details/').search(href)

for sect in sections:
    with requests.Session() as session:
        response = session.get(sect, headers = req_headers)
    soup = BeautifulSoup(response.content,'html.parser')

    home_links = []
    for a in soup.find_all(href = homes):
        print(a.get('href'))
        home_links.append('www.zillow.com' + a.get('href'))


In [74]:
print(sections)

[]


In [ ]:
i = 0
for link in home_links:
    
    with requests.Session() as session:
        response = session.get(link, headers = request_headers)
    soup = BeautifulSoup(response.content,'html.parser')
    try:
        add_span = soup.find_all(id = 'ds-chip-property-address')[0]
    except IndexError:
        pass
    address = " ".join(add_span.text.split())

    for summary in soup.find_all(class_ = "ds-summary-row"):
        try: 
            sum_dat = summary.find_all('span')
            price_cont = sum_dat[2].get_text()
            bd_cont = sum_dat[5].get_text()
            ba_cont = sum_dat[9].get_text()
            sq_cont = sum_dat[13].get_text()
        except IndexError:
            pass
    
    new_row = pd.Series(data = {'link': link, 'address': address, 'list_price': price_cont, 'beds': bd_cont, 'baths': ba_cont, 'sqft': sq_cont}, name = i)
    df = df.append(new_row, ignore_index=False)
    i += 1
print(df.head())